In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA

In [2]:
link = 'http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
df = pd.read_csv(link, header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [3]:
df = df.replace('?', np.NaN)
df.iloc[:,1] = df.iloc[:,1].astype(float)
df.info()
print('\nNans in each column:')
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     678 non-null object
1     678 non-null float64
2     690 non-null float64
3     684 non-null object
4     684 non-null object
5     681 non-null object
6     681 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    677 non-null object
14    690 non-null int64
15    690 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 56.6+ KB

Nans in each column:
0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64


In [4]:
df.describe(include= 'all')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,678,678.000000,690.000000,684,684,681,681,690.000000,690,690,690.00000,690,690,677,690.000000,690
unique,2,NaN,NaN,3,3,14,9,NaN,2,2,NaN,2,3,170,NaN,2
top,b,NaN,NaN,u,g,c,v,NaN,t,f,NaN,f,g,00000,NaN,-
freq,468,NaN,NaN,519,519,137,399,NaN,361,395,NaN,374,625,132,NaN,383
mean,NaN,31.568171,4.758725,NaN,NaN,NaN,NaN,2.223406,NaN,NaN,2.40000,NaN,NaN,NaN,1017.385507,NaN
std,NaN,11.957862,4.978163,NaN,NaN,NaN,NaN,3.346513,NaN,NaN,4.86294,NaN,NaN,NaN,5210.102598,NaN
min,NaN,13.750000,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,22.602500,1.000000,NaN,NaN,NaN,NaN,0.165000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
50%,NaN,28.460000,2.750000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.00000,NaN,NaN,NaN,5.000000,NaN
75%,NaN,38.230000,7.207500,NaN,NaN,NaN,NaN,2.625000,NaN,NaN,3.00000,NaN,NaN,NaN,395.500000,NaN


In [5]:
import ml_nechai as ml

#### Dropping data

In [6]:
df1 = ml.dropna(df, subset= [1,2,7,10,14])
# Get targets
targets = df1[15].astype('category')
# Save target-values as color for plotting
# red: -,  green: +
label_color = ['red' if i == '-' else 'green' for i in targets]
print(label_color[:3], label_color[-3:])

['green', 'green', 'green'] ['red', 'red', 'red']


In [7]:
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df1[[1,2,7,10,14]], targets, test_size=0.2, random_state=0)
# Create classifier
lr = LogisticRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
accuracy_score(y_test, y_pred)

c:\users\vlad nechai\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7205882352941176

#### Prepocessing

In [8]:
df = ml.normalize(df, columns = [2, 7, 10])
df = ml.standardize(df, columns = [1, 10, 14])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,0.256842,-0.955920,u,g,w,v,-0.290872,t,t,0.014925,f,g,00202,0.00000,+
1,a,0.675489,-0.060007,u,g,q,h,0.244013,t,t,0.089552,f,g,00043,0.00560,+
2,a,0.161654,-0.855481,u,g,q,h,-0.216167,t,f,0.000000,f,g,00280,0.00824,+
3,b,0.211729,-0.646569,u,g,w,v,0.456175,t,t,0.074627,t,g,00100,0.00003,+
4,b,0.096541,0.174015,u,g,w,v,-0.153415,t,f,0.000000,f,s,00120,0.00000,+


In [9]:
df.describe(include= 'all')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,678,678.000000,6.900000e+02,684,684,681,681,6.900000e+02,690,690,690.000000,690,690,677,690.000000,690
unique,2,NaN,NaN,3,3,14,9,NaN,2,2,NaN,2,3,170,NaN,2
top,b,NaN,NaN,u,g,c,v,NaN,t,f,NaN,f,g,00000,NaN,-
freq,468,NaN,NaN,519,519,137,399,NaN,361,395,NaN,374,625,132,NaN,383
mean,NaN,0.267942,1.029772e-17,NaN,NaN,NaN,NaN,1.029772e-16,NaN,NaN,0.035821,NaN,NaN,NaN,0.010174,NaN
std,NaN,0.179817,1.000000e+00,NaN,NaN,NaN,NaN,1.000000e+00,NaN,NaN,0.072581,NaN,NaN,NaN,0.052101,NaN
min,NaN,0.000000,-9.559198e-01,NaN,NaN,NaN,NaN,-6.643947e-01,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,0.133120,-7.550425e-01,NaN,NaN,NaN,NaN,-6.150897e-01,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN
50%,NaN,0.221203,-4.035072e-01,NaN,NaN,NaN,NaN,-3.655762e-01,NaN,NaN,0.000000,NaN,NaN,NaN,0.000050,NaN
75%,NaN,0.368120,4.919034e-01,NaN,NaN,NaN,NaN,1.200038e-01,NaN,NaN,0.044776,NaN,NaN,NaN,0.003955,NaN


_Displaying Nans in 1st col_

In [10]:
df[pd.isnull(df[1])]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
83,a,NaN,-0.252849,u,g,d,v,0.232061,t,f,0.000000,t,g,00300,0.00000,-
86,b,NaN,-0.880591,u,g,d,v,-0.402929,t,f,0.000000,t,s,00928,0.00000,-
92,b,NaN,0.048467,y,p,aa,v,1.875562,t,f,0.000000,f,g,00000,0.00000,-
97,b,NaN,-0.855481,u,g,c,bb,-0.414881,t,f,0.000000,t,s,00320,0.00000,-
254,b,NaN,-0.830371,u,g,k,v,-0.589690,f,f,0.000000,f,g,00380,0.02010,-
286,a,NaN,-0.654604,u,g,ff,ff,-0.664395,f,t,0.029851,t,g,00200,0.00105,-
329,b,NaN,-0.152411,y,p,i,v,-0.638995,f,f,0.000000,t,g,00411,0.00000,-
445,a,NaN,1.303950,u,g,ff,ff,-0.664395,f,f,0.000000,f,g,NaN,0.05200,-
450,b,NaN,-0.353288,y,p,i,bb,1.427335,f,f,0.000000,f,g,00000,0.00001,-
500,b,NaN,-0.152411,u,g,x,v,0.829698,t,t,0.044776,t,g,00290,0.02279,+


#### First is KNN

In [11]:
df1 = ml.replacenaKNN(df, 1, train = [2,7,10])

In [12]:
df1.iloc[[83, 86, 92, 97, 254, 286, 329, 445, 450, 500, 515, 608]]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
83,a,0.173143,-0.252849,u,g,d,v,0.232061,t,f,0.000000,t,g,00300,0.00000,-
86,b,0.150105,-0.880591,u,g,d,v,-0.402929,t,f,0.000000,t,s,00928,0.00000,-
92,b,0.169654,0.048467,y,p,aa,v,1.875562,t,f,0.000000,f,g,00000,0.00000,-
97,b,0.249143,-0.855481,u,g,c,bb,-0.414881,t,f,0.000000,t,s,00320,0.00000,-
254,b,0.257895,-0.830371,u,g,k,v,-0.589690,f,f,0.000000,f,g,00380,0.02010,-
286,a,0.230316,-0.654604,u,g,ff,ff,-0.664395,f,t,0.029851,t,g,00200,0.00105,-
329,b,0.181925,-0.152411,y,p,i,v,-0.638995,f,f,0.000000,t,g,00411,0.00000,-
445,a,0.179188,1.303950,u,g,ff,ff,-0.664395,f,f,0.000000,f,g,NaN,0.05200,-
450,b,0.263398,-0.353288,y,p,i,bb,1.427335,f,f,0.000000,f,g,00000,0.00001,-
500,b,0.228301,-0.152411,u,g,x,v,0.829698,t,t,0.044776,t,g,00290,0.02279,+


In [13]:
df1 = ml.replacena(df1, subset= [0,3,4,5,6,13], mode = 'mode')

In [14]:
df1.info()
print('\nNans in each column:')
print(df1.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null float64
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null float64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null float64
15    690 non-null object
dtypes: float64(5), object(11)
memory usage: 56.6+ KB

Nans in each column:
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


In [15]:
# Get targets
targets = df1[15].astype('category')
# Save target-values as color for plotting
# red: -,  green: +
label_color = ['red' if i == '-' else 'green' for i in targets]
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df1[[1,2,7,10,14]], targets, test_size=0.2, random_state=0)
# Create classifier
lr = LogisticRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
accuracy_score(y_test, y_pred)

c:\users\vlad nechai\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6739130434782609

Accuracy lost |x_x|    
Trying something else

#### Mean replace

In [16]:
df1 = ml.replacena(df, subset=[1])

In [17]:
df1 = ml.replacena(df1, subset= [0,3,4,5,6,13], mode = 'mode')

In [18]:
# Get targets
targets = df1[15].astype('category')
# Save target-values as color for plotting
# red: -,  green: +
label_color = ['red' if i == '-' else 'green' for i in targets]
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df1[[1,2,7,10,14]], targets, test_size=0.2, random_state=0)
# Create classifier
lr = LogisticRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
accuracy_score(y_test, y_pred)

c:\users\vlad nechai\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6884057971014492

Just a little better than before

#### Median replace

In [19]:
df1 = ml.replacena(df, subset=[1], mode = 'median')

In [20]:
df1 = ml.replacena(df1, subset= [0,3,4,5,6,13], mode = 'mode')

In [21]:
# Get targets
targets = df1[15].astype('category')
# Save target-values as color for plotting
# red: -,  green: +
label_color = ['red' if i == '-' else 'green' for i in targets]
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df1[[1,2,7,10,14]], targets, test_size=0.2, random_state=0)
# Create classifier
lr = LogisticRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
accuracy_score(y_test, y_pred)

c:\users\vlad nechai\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6739130434782609

Became even worse =(

#### Linear regression

In [22]:
df1 = ml.replacena_lr(df, 1)

In [23]:
df1 = ml.replacena(df1, subset= [0,3,4,5,6,13], mode = 'mode')

In [24]:
# Get targets
targets = df1[15].astype('category')
# Save target-values as color for plotting
# red: -,  green: +
label_color = ['red' if i == '-' else 'green' for i in targets]
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df1[[1,2,7,10,14]], targets, test_size=0.2, random_state=0)
# Create classifier
lr = LogisticRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
accuracy_score(y_test, y_pred)

c:\users\vlad nechai\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6884057971014492

Accuracy slightly increased compared to KNN, still less than dropping